[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/swati-rajwal/BreastCancer_tweets_project/blob/main/D_Treatment_Topic_Identification_Word2Vec_Roberta_Self_Report.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install bertopic

In [ ]:
# Import Libraries
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
from bertopic import BERTopic
import matplotlib.pyplot as plt
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Import Self Classified (Roberta Classifier : 1) tweets dataset
df = pd.read_csv('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/CSV Files/roberta_prediction_1_only_with_sentiment.csv')
print("df original shape:", df.shape)

# Load pre-trained Word2Vec embeddings
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/GoogleNews-vectors-negative300.bin', binary=True)

# Filter Tweets with negative sentiments
df = df[df['sentiment_on_preprocessed'] <= 0.5].copy()
df = df.reset_index(drop=True)

# Filter posts that contain treatment side-effect / negative keywords
trestment_bc_negative_keywords = ["treatment","medication","medicine","med","tablets","side effect","reaction","drug","tamoxifen","chemo","mental","emotion",
                     "lump","pain","sleep","docetaxel","oncologist","doc" "stop"]  # Keywords based on literature review of Pub_med papers (reference in main article)

# Function to find similar words in a text
def find_similar_words(text, word_vectors, uncertainty_words, threshold=0.5):
    similar_words = []
    unc_words = []
    for word in text.lower().split():
        if word in word_vectors:
            word_embedding = word_vectors[word]
            for unc_word in uncertainty_words:
                if unc_word in word_vectors:
                    unc_embedding = word_vectors[unc_word]
                    similarity = cosine_similarity([word_embedding], [unc_embedding])[0][0]
                    if similarity > threshold:
                        similar_words.append(word)
                        similar_words.append(unc_word)
                        break
    return similar_words

# Load the dataset
df['similar_uncertainty_words'] = df['text'].apply(lambda x: find_similar_words(x, word_vectors, trestment_bc_negative_keywords))

df original shape: (154571, 5)


In [ ]:
df.to_csv('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/CSV Files/roberta_prediction_1_only_with_sentiment_similar_words.csv', index=False)

In [ ]:
# Extracting tweets of self reported BC with negative sentiments and matched word2vec from treatment side effect keywords
df = df[df['similar_uncertainty_words'].str.len() > 0].copy()
self_reported_texts = df['original_text'].to_list()
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(self_reported_texts)

# Visualize Topic Modelling Figure
topic_model.visualize_barchart(top_n_topics=70)


2024-03-31 18:24:28,207 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2990 [00:00<?, ?it/s]

2024-03-31 18:36:46,791 - BERTopic - Embedding - Completed ✓
2024-03-31 18:36:46,794 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-31 18:38:34,774 - BERTopic - Dimensionality - Completed ✓
2024-03-31 18:38:34,778 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-31 18:38:44,263 - BERTopic - Cluster - Completed ✓
2024-03-31 18:38:44,301 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-31 18:38:50,006 - BERTopic - Representation - Completed ✓
